In [1]:
# Give access to utils
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import util

In [2]:
# Load an example publication returned
with open('nature07365.html', 'r') as example_html:
    html = example_html.read()

In [3]:
# Import into Beautiful Soup
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(html, 'html.parser')

# Find article sections
sections = soup.findAll('section', {'aria-labelledby': re.compile(r".*")})
len(sections)

7

In [6]:
# We define a method that turns HTML into text
# Source: https://stackoverflow.com/questions/328356/extracting-text-from-html-file-using-python
def html2txt(html):
    soup = BeautifulSoup(html, 'html.parser')

    # Kill all script, style, and link elements
    for script in soup(['script', 'style', 'a', 'h2']):
        script.extract()    # rip it out
        
    # Get text
    text = soup.get_text()
    
    return text

In [7]:
import re

# Certain sections contain useless information
skip_sections = set([
    'acknowledgements',
    'references',
    'author-information',
    'supplementary-information',
    'article-comments'
])

section_texts = []

# Turn each section that does have good info into text
for section in sections:
    
    # Skip useless sections
    s_title = section['aria-labelledby']
    if s_title in skip_sections:
        continue
    
    # Otherwise we turn the section into plain text
    text = html2txt(unicode(section))
    text = re.sub(r'[^\x00-\x7f]',r' ', text) # remove non-unicode
    section_texts.append(text)
    

In [8]:
texts = '\n'.join(section_texts)
#print texts

Certain forums have recommended number substition - could be worth looking into. Just tokens for negative, 0-1, and positive numbers. 

Additionally we want to split by everything that is not a letter

In [9]:
def is_number(s):
    return s[0].isdigit()

# Check if numbers can be identified
is_number('18Ocal')

True

In [44]:
import re

# Split the string by space first
texts_word_lists = []
for text in section_texts:
    text_list = text.split(' ')
    text_list_out = []
    for word in text_list:
        if not word:
            continue
        elif is_number(word):
            text_list_out.append('TOKEN_NUMBER')
        else:
            subwords = re.split('[^a-zA-Z]', word)
            text_list_out.extend(filter(None, subwords))
    texts_word_lists.append(text_list_out)
    
texts_word_lists

[[u'Climate',
  u'in',
  u'the',
  u'early',
  u'Pleistocene',
  u'varied',
  u'with',
  u'a',
  u'period',
  u'of',
  'TOKEN_NUMBER',
  u'kyr',
  u'and',
  u'was',
  u'related',
  u'to',
  u'variations',
  u'in',
  u'Earth',
  u's',
  u'obliquity',
  u'About',
  'TOKEN_NUMBER',
  u'kyr',
  u'ago',
  u'variability',
  u'increased',
  u'and',
  u'oscillated',
  u'primarily',
  u'at',
  u'a',
  u'period',
  u'of',
  'TOKEN_NUMBER',
  u'kyr',
  u'suggesting',
  u'that',
  u'the',
  u'link',
  u'was',
  u'then',
  u'with',
  u'the',
  u'eccentricity',
  u'of',
  u'Earth',
  u's',
  u'orbit',
  u'This',
  u'transition',
  u'has',
  u'often',
  u'been',
  u'attributed',
  u'to',
  u'a',
  u'nonlinear',
  u'response',
  u'to',
  u'small',
  u'changes',
  u'in',
  u'external',
  u'boundary',
  u'conditions',
  u'Here',
  u'we',
  u'propose',
  u'that',
  u'increasing',
  u'variablility',
  u'within',
  u'the',
  u'past',
  u'million',
  u'years',
  u'may',
  u'indicate',
  u'that',
  u'the',
 

In [43]:
section_texts[0]

u"Climate in the early Pleistocene varied with a period of 41 kyr and was related to variations in Earth's obliquity. About 900 kyr ago, variability increased and oscillated primarily at a period of  100 kyr, suggesting that the link was then with the eccentricity of Earth's orbit. This transition has often,,, been attributed to a nonlinear response to small changes in external boundary conditions. Here we propose that increasing variablility within the past million years may indicate that the climate system was approaching a second climate bifurcation point, after which it would transition again to a new stable state characterized by permanent mid-latitude Northern Hemisphere glaciation. From this perspective the past million years can be viewed as a transient interval in the evolution of Earth's climate. We support our hypothesis using a coupled energy-balance/ice-sheet model, which furthermore predicts that the future transition would involve a large expansion of the Eurasian ice sh